In [1]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=3

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end
function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
v+=star_algebra.(v)
v=v./2
f=generate_random_poly(v)


# unit sphere constraint
m=n
q=floor(Int64,n/m)
R=ones(Float64,m)./n
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

h=Vector{Polynomial{false,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=3
Number of inequality constraints: m=3
Number of equality constraints: l=1


In [2]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [3]:
k=2
println("Relaxed order: k=",k)

Relaxed order: k=2


In [4]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-4,
                                 use_eqcons_to_get_constant_trace=false,
                                 check_tol_each_iter=true)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1


  Computing constant trace status: OPTIMAL
  Constant trace: ak = 2.9999999999999996
  Number of blocks: omega=4
  Size of the largest block: s^max=13
  Number of equality trace constraints: zeta=59
Modeling time:
 22.643110 seconds (47.69 M allocations: 2.341 GiB, 6.34% gc time)
iter=1   val=-2.9337248060116368   gap=0.008278432771066753   feas=3.8679044086554337
iter=2   val=-0.7472735650308815   gap=0.36643295845590323   feas=1.2494448508985363
iter=4   val=-0.7961716021403527   gap=0.24689461586687234   feas=1.2902882551710395
iter=8   val=-0.26790291133538413   gap=0.01816262645910669   feas=0.794545028397422
iter=16   val=0.22654883180875005   gap=0.037160430600511093   feas=0.3399857777928634
iter=32   val=0.3525888795523007   gap=0.08695172537920554   feas=0.15727906838631037
iter=64   val=0.3817172647666482   gap=0.03650598785066348   feas=0.07028823542653685
iter=128   val=0.41267432784271013   gap=0.03135474837016494   feas=0.02385476311853516
iter=256   val=0.41976118092374

0.42473794709583407

In [5]:
using NCTSSOS

@time ctpNCPOP.nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
    n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS=false,QUIET=false);

------------------------------------------------------
The sizes of blocks:
[13]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 73              
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Probl